In [1]:
!pip install catboost

In [2]:
import numpy as np
import pandas as pd

import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer, f1_score, accuracy_score, roc_auc_score
%matplotlib inline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

### Dataset: Job Applicants Data (Human Resource)


In [10]:
df2_bin = pd.read_csv('job_bin.csv') #previosly bionirize
df2_bin.drop(['Unnamed: 0'], axis=1, inplace = True)
df2_bin

,Employment,YearsCode,YearsCodePro,PreviousSalary,ComputerSkills,Employed,Age_>35,Accessibility_Yes,EdLevel_NoHigherEd,EdLevel_Other,...,EdLevel_Undergraduate,Gender_NonBinary,Gender_Woman,MentalHealth_Yes,MainBranch_NotDev,Employment_bin,YearsCode_bin,YearsCodePro_bin,PreviousSalary_bin,ComputerSkills_bin
0,True,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,True,True,True,True,True,True,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
2,True,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,True,True,False,True,False
3,True,True,True,True,True,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,True
4,False,True,True,True,True,False,True,False,False,False,...,False,False,False,False,True,False,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,True,True,True,True,True,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
996,True,True,True,True,True,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
997,False,True,True,True,True,True,True,False,False,False,...,False,False,False,False,False,False,True,True,True,True
998,True,True,True,True,True,False,True,False,False,False,...,False,False,False,False,False,True,True,True,True,False


In [11]:
y_feat = 'Employed'
df_train, df_test = train_test_split(df2_bin, train_size=0.7, random_state=0)
X_train, y_train = df_train.drop(y_feat, axis=1), df_train[y_feat]
X_test, y_test = df_test.drop(y_feat, axis=1), df_test[y_feat]
X_train.describe()

,Employment,YearsCode,YearsCodePro,PreviousSalary,ComputerSkills,Age_>35,Accessibility_Yes,EdLevel_NoHigherEd,EdLevel_Other,EdLevel_PhD,EdLevel_Undergraduate,Gender_NonBinary,Gender_Woman,MentalHealth_Yes,MainBranch_NotDev,Employment_bin,YearsCode_bin,YearsCodePro_bin,PreviousSalary_bin,ComputerSkills_bin
count,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700,700
unique,2,1,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
freq,604,700,677,700,700,490,683,668,591,672,373,691,678,571,627,604,422,450,419,377


In [12]:
X_test.describe()

,Employment,YearsCode,YearsCodePro,PreviousSalary,ComputerSkills,Age_>35,Accessibility_Yes,EdLevel_NoHigherEd,EdLevel_Other,EdLevel_PhD,EdLevel_Undergraduate,Gender_NonBinary,Gender_Woman,MentalHealth_Yes,MainBranch_NotDev,Employment_bin,YearsCode_bin,YearsCodePro_bin,PreviousSalary_bin,ComputerSkills_bin
count,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300,300
unique,2,1,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True
freq,263,300,291,300,300,197,288,289,258,283,151,296,290,244,270,263,172,172,168,151


In [13]:
# bool -> 0, 1
X_train = X_train.astype(int)
X_test = X_test.astype(int)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [14]:
results = {
    'Model': [],
    'Accuracy': [],
    'F1 Score': [],
}

models = {
    'KNN': KNeighborsClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
}

params = {
    'KNN': {'n_neighbors': [3, 5, 7, 10]},
    'DecisionTree': {'max_depth': [None, 5, 7, 10]},
    'RandomForest': {'n_estimators': [10, 20, 35, 50], 'max_depth': [None, 2, 5, 10]},
    'LogisticRegression': {'C': [0.1, 1, 10]},
}

In [15]:
for name, model in models.items():
    clf = GridSearchCV(model, params[name], cv=5, scoring=make_scorer(f1_score, average='macro'))
    clf.fit(X_train, y_train)

    acc_scores = cross_val_score(clf.best_estimator_, X_test, y_test, cv=5, scoring='accuracy')
    f1_scores = cross_val_score(clf.best_estimator_, X_test, y_test, cv=5, scoring=make_scorer(f1_score, average='macro'))

    results['Model'].append(name)
    results['Accuracy'].append(round(acc_scores.mean(),3))
    results['F1 Score'].append(round(f1_scores.mean(),3))

df_results = pd.DataFrame(results)
print(df_results)

                Model  Accuracy  F1 Score
0                 KNN     0.783     0.782
1        DecisionTree     0.783     0.781
2        RandomForest     0.810     0.805
3  LogisticRegression     0.820     0.818
